## Data Exploration and Cleaning with SQL

### Importing Database

In [1]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

# We are also going to do some basic viz
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:

# There is a bug in matplotlib. You cannot set the rc parameters in the same
# cell that you use the "%matplotlib inline" magic command
plt.style.use('ggplot')
plt.rc('font', size=18)

In [2]:
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'juliaqiao',    # DB that we are connecting to
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [3]:
#load our full dataset
query = "SELECT * FROM nls;"

pd_sql.read_sql(query, connection)

,id,sex,birth_year,sample_type,race,poverty,degree_2017,marriage,metro_area,child_at_home,...,income_wage_est,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,house_type_35,fin_assets_35,debts_35
0,1,2,1981,1,4,-5,-5,-5,-5,-5,...,-5,-5,-5,4,6,30000,18000,-4,-4,-4
1,2,1,1982,1,2,929,2,2,2,-4,...,-4,-4,-4,2,6,45000,35000,1,2000,62500
2,3,2,1983,1,2,-3,4,3,2,2,...,-1,-4,-4,4,6,-3,19300,-4,-4,-4
3,4,2,1981,1,2,179,2,2,3,1,...,-4,-4,-4,2,6,0,0,6,7050,1300
4,5,1,1982,1,2,871,2,3,3,2,...,-4,-4,-4,2,1,144000,9000,1,300000,3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8979,9018,2,1980,1,4,327,1,3,2,0,...,-4,-4,-4,1,6,100,1200,1,4000,19000
8980,9019,1,1984,0,2,498,3,2,3,0,...,-4,-4,-4,3,6,400,17000,-4,-4,-4
8981,9020,1,1980,1,4,-5,-5,-5,-5,-5,...,-5,-5,-5,2,6,6000,0,6,78750,0
8982,9021,1,1980,1,4,260,4,2,2,1,...,-4,-4,-4,4,6,5000,0,6,9000,0


### Exploration and Cleaning

In [10]:
#counting house ownership count by category, for age 30
query_house_count_30 = """
        SELECT house_type_30, COUNT(house_type_30)
        FROM nls
        GROUP BY house_type_30
        ORDER BY COUNT(house_type_30) DESC 
"""

pd_sql.read_sql(query_house_count_30, connection)

,house_type_30,count
0,6,5352
1,1,1936
2,-4,1274
3,9,247
4,3,156
5,-3,17
6,2,2


CVC_HOUSE_TYPE_30 KEY
  1='House'
  2='Ranch/Farm'
  3='Mobile Home'
  6='R does not own'
  9='Owns other residence type'

In [11]:
#counting house ownership count by category, for age 35
query_house_count_35 = """
        SELECT house_type_35, COUNT(house_type_35)
        FROM nls
        GROUP BY house_type_35
        ORDER BY COUNT(house_type_35) DESC 
"""

pd_sql.read_sql(query_house_count_35, connection)

,house_type_35,count
0,-4,4817
1,6,2360
2,1,1617
3,9,128
4,3,46
5,-3,14
6,2,2


Interview Skip Key:

Noninterview -5

Valid Skip -4

Invalid Skip -3

Don't Know -2

Refusal -1

(from the nls_key_decoder_sas)

Between age 30 and 35, it's clear that we should stick to 30 for a larger sample size, as most of the age 35 answers were valid skips (since only half of the subjects had reached 35 by the last survey).

In [12]:
#select rows where house count is not a skip
query_house_30 = """
        SELECT *
        FROM nls
        WHERE house_type_30 > 0
"""

clean_0 = pd_sql.read_sql(query_house_30, connection)

In [14]:
#df with valid y variables-- house ownership answers
#let's pickle it as our first clean df, iteration 0.
clean_0.to_pickle('clean_0.pickle')
clean_0

,id,sex,birth_year,sample_type,race,poverty,degree_2017,marriage,metro_area,child_at_home,...,income_wage_est,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,house_type_35,fin_assets_35,debts_35
0,1,2,1981,1,4,-5,-5,-5,-5,-5,...,-5,-5,-5,4,6,30000,18000,-4,-4,-4
1,2,1,1982,1,2,929,2,2,2,-4,...,-4,-4,-4,2,6,45000,35000,1,2000,62500
2,3,2,1983,1,2,-3,4,3,2,2,...,-1,-4,-4,4,6,-3,19300,-4,-4,-4
3,4,2,1981,1,2,179,2,2,3,1,...,-4,-4,-4,2,6,0,0,6,7050,1300
4,5,1,1982,1,2,871,2,3,3,2,...,-4,-4,-4,2,1,144000,9000,1,300000,3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7688,9018,2,1980,1,4,327,1,3,2,0,...,-4,-4,-4,1,6,100,1200,1,4000,19000
7689,9019,1,1984,0,2,498,3,2,3,0,...,-4,-4,-4,3,6,400,17000,-4,-4,-4
7690,9020,1,1980,1,4,-5,-5,-5,-5,-5,...,-5,-5,-5,2,6,6000,0,6,78750,0
7691,9021,1,1980,1,4,260,4,2,2,1,...,-4,-4,-4,4,6,5000,0,6,9000,0


In [28]:
#let's see if we have any NaNs
clean_0.isnull().sum()

id                   0
sex                  0
birth_year           0
sample_type          0
race                 0
poverty              0
degree_2017          0
marriage             0
metro_area           0
child_at_home        0
urban_rural          0
income_wage_total    0
income_wage_est      0
income_bus_total     0
income_bus_est       0
degree_other         0
house_type_30        0
fin_assets_30        0
debts_30             0
house_type_35        0
fin_assets_35        0
debts_35             0
dtype: int64

In [34]:
#let's now look at our duplicate feature--degrees, to see which one is more recent!
query_degree = """
        SELECT degree_2017, degree_other
        FROM nls 
        WHERE house_type_30 NOT IN (-5,-4,-3,-2,-1)
        AND degree_2017 != degree_other
        
        
"""

pd_sql.read_sql(query_degree, connection)

,degree_2017,degree_other
0,-5,4
1,-5,1
2,-5,5
3,-5,1
4,-5,2
...,...,...
1059,-5,2
1060,-5,2
1061,-5,2
1062,-5,4


In [35]:
#looks like degree_2017 has a lot of skipped answers - noninterview. Let's see how many skips degrees_2017 
query_degree_2017 = """
        SELECT degree_2017, COUNT(degree_2017)
        FROM nls 
        WHERE house_type_30 NOT IN (-5,-4,-3,-2,-1)
        AND degree_2017 IN (-5,-4,-3,-2,-1)
        GROUP BY degree_2017
        
        
"""

pd_sql.read_sql(query_degree_2017, connection)


,degree_2017,count
0,-5,1064
1,-3,26


In [36]:
#how about compared to degrees_other?
query_degree_other = """
        SELECT degree_other, COUNT(degree_other)
        FROM nls 
        WHERE house_type_30 NOT IN (-5,-4,-3,-2,-1)
        AND degree_other IN (-5,-4,-3,-2,-1)
        GROUP BY degree_other      
        
"""

pd_sql.read_sql(query_degree_other, connection)

,degree_other,count
0,-3,30


Degree_other have much fewer skips than degree_2017 -- potentially a more recent variable, so we will stick with degree_other as the education feature!

In [9]:
#let's see how many rows we're left with if we drop all the interview skips for every relevant features aside from income (which we have to feature engineer)
#we'll also leave out the 35 columns as we know we won't be using them.
query_drop_all_skips = """
        SELECT id, sex, birth_year, sample_type, race, poverty, marriage, metro_area, child_at_home, income_wage_total, income_wage_est, income_bus_total, income_bus_est, degree_other, house_type_30, fin_assets_30, debts_30
        FROM nls 
        WHERE house_type_30 NOT IN (-5,-4,-3,-2,-1) 
        AND degree_other NOT IN (-5,-4,-3,-2,-1)
        AND poverty NOT IN (-5,-4,-3,-2,-1)
        AND marriage NOT IN (-5,-4,-3,-2,-1)
        AND metro_area NOT IN (-5,-4,-3,-2,-1)
        AND child_at_home NOT IN (-5,-4,-3,-2,-1)
        AND fin_assets_30 NOT IN (-5,-4,-3,-2,-1)
        AND debts_30 NOT IN (-5,-4,-3,-2,-1)     
        
"""

clean_no_skips = pd_sql.read_sql(query_drop_all_skips, connection)

In [10]:
clean_no_skips.to_pickle('clean_no_skips.pickle')
clean_no_skips

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,income_wage_total,income_wage_est,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30
0,4,2,1981,1,2,179,2,3,1,45000,-4,-4,-4,2,6,0,0
1,5,1,1982,1,2,871,3,3,2,125000,-4,-4,-4,2,1,144000,9000
2,6,2,1982,1,2,226,3,3,2,-2,3,-4,-4,2,6,3500,1000
3,10,1,1984,1,4,906,3,3,1,75000,-4,-4,-4,4,6,185000,96500
4,11,2,1982,1,2,399,3,3,1,36000,-4,-4,-4,4,6,3000,62500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773,9013,1,1982,0,1,140,2,3,0,-2,3,-4,-4,1,6,1750,0
3774,9015,2,1980,0,1,325,3,2,3,56000,-4,-4,-4,2,1,88176,55464
3775,9018,2,1980,1,4,327,3,2,0,17500,-4,-4,-4,1,6,100,1200
3776,9019,1,1984,0,2,498,2,3,0,35000,-4,-4,-4,3,6,400,17000


In [151]:
clean_no_skips = pd.read_pickle('clean_no_skips.pickle')

### Feature Engineering

Great, now let's feature engineer our income columns. We have 4 columns--estimated and actual for salary/wages and for business/farm income. let's combine them into 2 encoded columns of estimated salary--one for wages and one for business/farm.

In [152]:
clean_no_skips.income_wage_est.value_counts()

-4    3600
 3      58
 4      44
 2      30
 1      24
 5      16
 6       5
 7       1
Name: income_wage_est, dtype: int64

Income Wage Code key: 
  1='A. $1 - $5,000'
  2='B. $5,001 - $10,000'
  3='C. $10,001 - $25,000'
  4='D. $25,001 - $50,000'
  5='E. $50,001 - $100,000'
  6='F. $100,001 - $250,000'
  7='G. More than $250,000'

In [153]:
def income_wage_encoder(total):
    '''Takes in a continuous wage income value and groups it into an income interval represented by an integer value
    '''
    if total <= 0:
        return 0
    if 1< total <= 5000:
        return 1
    if 5000< total <= 10000:
        return 2
    if 10000< total <= 25000:
        return 3
    if 25000< total <= 50000:
        return 4
    if 50000< total <= 100000:
        return 5
    if 100000< total <= 250000:
        return 6
    elif total > 250000:
        return 7
      

In [154]:
#create new column to store these coded wage income values.
clean_no_skips['income_wage_total_coded'] = clean_no_skips.apply(lambda x: income_wage_encoder(x['income_wage_total']),axis=1)

Original Business/farm income encoder from source data:
  1='A.   LOST/WOULD LOSE MONEY'
  2='B.   $1              -        $5,000'
  3='C.   $5,001       -        $10,000'
  4='D.   $10,001     -        $25,000'
  5='E.    $25,001     -        $50,000'
  6='F.    $50,001     -        $100,000'
  7='G.    $100,001   -        $250,000'
  8='H.    More than  $250,000'

TRANSFORMED Business/Farm income encoder (aligns with wage encoder)
  0='A.   LOST/WOULD LOSE MONEY'
  1='B.   $1              -        $5,000'
  2='C.   $5,001       -        $10,000'
  3='D.   $10,001     -        $25,000'
  4='E.    $25,001     -        $50,000'
  5='F.    $50,001     -        $100,000'
  6='G.    $100,001   -        $250,000'
  7='H.    More than  $250,000'

In [155]:
def income_bus_transformer(code):
    '''Takes in an encoded business/farm income integer value which represents an income interval and standardizes it to the same encoding as the wage income encoding '''
    if code > 0:
        code = code -1
        return code
    else:
        return 0

In [156]:
def income_bus_encoder(total):
    '''Takes in a continuous business/farm income value and groups it into an income interval represented by an integer value
    '''
    if total <= 0:
        return 0
    if 1< total <= 5000:
        return 1
    if 5000< total <= 10000:
        return 2
    if 10000< total <= 25000:
        return 3
    if 25000< total <= 50000:
        return 4
    if 50000< total <= 100000:
        return 5
    if 100000< total <= 250000:
        return 6
    elif total > 250000:
        return 7

In [157]:
#create new column to store these coded business/farm income values.
clean_no_skips['income_bus_total_coded'] = clean_no_skips.apply(lambda x: income_wage_encoder(x['income_bus_total']),axis=1)

In [158]:
#less than 3% of subjects make money from a business or farm. that passes our sanity check.
#our coded values for both total wage income and total business/farm income pass our sanity checks as well.
clean_no_skips[clean_no_skips.income_bus_total_coded.notnull()]

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,income_wage_total,income_wage_est,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,income_wage_total_coded,income_bus_total_coded
0,4,2,1981,1,2,179,2,3,1,45000,-4,-4,-4,2,6,0,0,4,0
1,5,1,1982,1,2,871,3,3,2,125000,-4,-4,-4,2,1,144000,9000,6,0
2,6,2,1982,1,2,226,3,3,2,-2,3,-4,-4,2,6,3500,1000,0,0
3,10,1,1984,1,4,906,3,3,1,75000,-4,-4,-4,4,6,185000,96500,5,0
4,11,2,1982,1,2,399,3,3,1,36000,-4,-4,-4,4,6,3000,62500,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773,9013,1,1982,0,1,140,2,3,0,-2,3,-4,-4,1,6,1750,0,0,0
3774,9015,2,1980,0,1,325,3,2,3,56000,-4,-4,-4,2,1,88176,55464,5,0
3775,9018,2,1980,1,4,327,3,2,0,17500,-4,-4,-4,1,6,100,1200,3,0
3776,9019,1,1984,0,2,498,2,3,0,35000,-4,-4,-4,3,6,400,17000,4,0


In [159]:
#change the original business/farm encoder into the transformed encoding version to match the wage encoder.
clean_no_skips['income_bus_est'] = clean_no_skips.apply(lambda x: income_bus_transformer(x['income_bus_est']),axis=1)

In [160]:
#looks great!
clean_no_skips[clean_no_skips.income_bus_est.notnull()]

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,income_wage_total,income_wage_est,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,income_wage_total_coded,income_bus_total_coded
0,4,2,1981,1,2,179,2,3,1,45000,-4,-4,0,2,6,0,0,4,0
1,5,1,1982,1,2,871,3,3,2,125000,-4,-4,0,2,1,144000,9000,6,0
2,6,2,1982,1,2,226,3,3,2,-2,3,-4,0,2,6,3500,1000,0,0
3,10,1,1984,1,4,906,3,3,1,75000,-4,-4,0,4,6,185000,96500,5,0
4,11,2,1982,1,2,399,3,3,1,36000,-4,-4,0,4,6,3000,62500,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773,9013,1,1982,0,1,140,2,3,0,-2,3,-4,0,1,6,1750,0,0,0
3774,9015,2,1980,0,1,325,3,2,3,56000,-4,-4,0,2,1,88176,55464,5,0
3775,9018,2,1980,1,4,327,3,2,0,17500,-4,-4,0,1,6,100,1200,3,0
3776,9019,1,1984,0,2,498,2,3,0,35000,-4,-4,0,3,6,400,17000,4,0


In [161]:
clean_no_skips.income_wage_total_coded.value_counts()

4    1115
0     872
5     786
3     522
6     217
2     137
1     129
Name: income_wage_total_coded, dtype: int64

In [162]:
clean_no_skips.income_wage_total.isnull().sum()

0

In [163]:
clean_no_skips.income_wage_est.value_counts()

-4    3600
 3      58
 4      44
 2      30
 1      24
 5      16
 6       5
 7       1
Name: income_wage_est, dtype: int64

In [164]:
clean_no_skips.income_wage_est.isnull().sum()

0

In [165]:
def income_compiler(coded_total, estimate):
    if estimate <=0:
        if coded_total > 0:
            return coded_total
        else: 
            return 0
    elif estimate >0:
        if coded_total  <= 0:
            return estimate
        else:
            return 1000
        #catch all incase both columns are encoded for the same row


In [166]:
#let's compile the newly coded totals and originally coded estimates now.
clean_no_skips['income_wage_compiled'] = clean_no_skips.apply(lambda x: income_compiler(x['income_wage_total_coded'],x['income_wage_est']),axis=1)

In [167]:
#our catchall didn't catch any columns that were double encoded.
#wage income columns are successfully compiled.
clean_no_skips[clean_no_skips['income_wage_compiled'] > 7]

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,income_wage_total,income_wage_est,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,income_wage_total_coded,income_bus_total_coded,income_wage_compiled


In [168]:
#a number of subjects didn't report any income from wages
#a number will report it in the next column we compile--for business/farm income
#the rest have refused to answer

clean_no_skips[clean_no_skips['income_wage_compiled'] == 0]

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,income_wage_total,income_wage_est,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,income_wage_total_coded,income_bus_total_coded,income_wage_compiled
6,14,1,1980,1,2,205,2,3,0,-4,-4,-4,0,1,6,0,0,0,0,0
13,25,2,1983,1,2,131,3,3,3,-4,-4,-4,0,1,6,0,0,0,0,0
15,36,2,1980,1,4,104,7,2,2,-4,-4,-4,0,3,6,5100,28570,0,0,0
17,44,1,1981,1,2,81,8,3,0,-4,-4,-4,0,2,6,0,1750,0,0,0
36,80,1,1981,1,1,40,2,3,0,-4,-4,-4,0,1,6,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3753,8980,2,1981,1,4,154,5,3,0,-4,-4,-4,0,0,6,0,0,0,0,0
3754,8981,2,1980,1,4,0,2,3,2,-4,-4,-4,0,2,6,1500,2880,0,0,0
3757,8988,2,1980,0,1,16,2,2,3,-4,-4,-4,0,1,6,0,0,0,0,0
3761,8994,2,1981,0,1,38,2,3,2,-4,-4,-4,0,2,6,2500,60,0,0,0


In [169]:
clean_no_skips.income_bus_total_coded.value_counts()

0    3665
3      32
1      27
2      20
4      18
5      13
6       3
Name: income_bus_total_coded, dtype: int64

In [170]:
clean_no_skips.income_bus_total_coded.isnull().sum()

0

In [171]:
clean_no_skips.income_bus_est.value_counts()

0    3768
3       4
1       2
5       1
6       1
4       1
2       1
Name: income_bus_est, dtype: int64

In [172]:
clean_no_skips.income_bus_est.isnull().sum()

0

In [173]:
#let's compile the newly coded totals and originally coded estimates now.
clean_no_skips['income_bus_compiled'] = clean_no_skips.apply(lambda x: income_compiler(x['income_bus_total_coded'],x['income_bus_est']),axis=1)

In [175]:
#our catchall didn't catch any columns that were double encoded.
#wage income columns are successfully compiled.
clean_no_skips[clean_no_skips['income_bus_compiled'] > 7]

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,income_wage_total,...,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,income_wage_total_coded,income_bus_total_coded,income_wage_compiled,income_bus_compiled


In [174]:
clean_no_skips[clean_no_skips['income_bus_compiled'] != 0]

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,income_wage_total,...,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,income_wage_total_coded,income_bus_total_coded,income_wage_compiled,income_bus_compiled
7,16,1,1982,1,2,240,2,3,0,20000,...,25000,0,3,6,3000,3000,3,3,3,3
21,49,2,1982,1,1,488,3,2,3,62000,...,6000,0,2,6,2500,0,5,2,5,2
111,311,1,1982,1,4,1248,3,2,1,68000,...,68000,0,4,1,55000,28500,5,5,5,5
120,357,1,1981,1,2,370,3,2,2,70000,...,20000,0,2,1,12500,5100,5,3,5,3
123,362,2,1981,1,2,165,8,3,4,-2,...,6000,0,2,6,7800,4300,0,2,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3555,8519,1,1982,0,1,560,1,2,0,26000,...,4000,0,2,6,0,0,4,1,4,1
3674,8792,2,1980,1,1,332,3,3,3,15000,...,15000,0,2,1,29500,23500,3,3,3,3
3681,8818,2,1980,1,2,93,8,3,4,-4,...,3120,0,2,6,0,0,0,1,0,1
3724,8922,2,1983,1,1,49,2,3,3,10000,...,2000,0,3,6,0,11750,2,1,2,1


In [186]:
#create a new dataframe where both compiled income columns have values
model_ready = clean_no_skips[(clean_no_skips['income_wage_compiled'] != 0) | (clean_no_skips['income_bus_compiled'] != 0)]
model_ready

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,income_wage_total,...,income_bus_total,income_bus_est,degree_other,house_type_30,fin_assets_30,debts_30,income_wage_total_coded,income_bus_total_coded,income_wage_compiled,income_bus_compiled
0,4,2,1981,1,2,179,2,3,1,45000,...,-4,0,2,6,0,0,4,0,4,0
1,5,1,1982,1,2,871,3,3,2,125000,...,-4,0,2,1,144000,9000,6,0,6,0
2,6,2,1982,1,2,226,3,3,2,-2,...,-4,0,2,6,3500,1000,0,0,3,0
3,10,1,1984,1,4,906,3,3,1,75000,...,-4,0,4,6,185000,96500,5,0,5,0
4,11,2,1982,1,2,399,3,3,1,36000,...,-4,0,4,6,3000,62500,4,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773,9013,1,1982,0,1,140,2,3,0,-2,...,-4,0,1,6,1750,0,0,0,3,0
3774,9015,2,1980,0,1,325,3,2,3,56000,...,-4,0,2,1,88176,55464,5,0,5,0
3775,9018,2,1980,1,4,327,3,2,0,17500,...,-4,0,1,6,100,1200,3,0,3,0
3776,9019,1,1984,0,2,498,2,3,0,35000,...,-4,0,3,6,400,17000,4,0,4,0


In [195]:
model_ready.columns

Index(['id', 'sex', 'birth_year', 'sample_type', 'race', 'poverty', 'marriage',
       'metro_area', 'child_at_home', 'income_wage_total', 'income_wage_est',
       'income_bus_total', 'income_bus_est', 'degree_other', 'house_type_30',
       'fin_assets_30', 'debts_30', 'income_wage_total_coded',
       'income_bus_total_coded', 'income_wage_compiled',
       'income_bus_compiled'],
      dtype='object')

In [196]:
#drop any columns we don't need during modeling
model_ready = model_ready.drop(columns =['income_wage_total', 'income_wage_est', 'income_bus_total', 
                                'income_bus_est', 'income_wage_total_coded', 
                                'income_bus_total_coded'])

In [197]:
model_ready

,id,sex,birth_year,sample_type,race,poverty,marriage,metro_area,child_at_home,degree_other,house_type_30,fin_assets_30,debts_30,income_wage_compiled,income_bus_compiled
0,4,2,1981,1,2,179,2,3,1,2,6,0,0,4,0
1,5,1,1982,1,2,871,3,3,2,2,1,144000,9000,6,0
2,6,2,1982,1,2,226,3,3,2,2,6,3500,1000,3,0
3,10,1,1984,1,4,906,3,3,1,4,6,185000,96500,5,0
4,11,2,1982,1,2,399,3,3,1,4,6,3000,62500,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773,9013,1,1982,0,1,140,2,3,0,1,6,1750,0,3,0
3774,9015,2,1980,0,1,325,3,2,3,2,1,88176,55464,5,0
3775,9018,2,1980,1,4,327,3,2,0,1,6,100,1200,3,0
3776,9019,1,1984,0,2,498,2,3,0,3,6,400,17000,4,0


In [198]:
#pickle it for our next notebook on modeling!
model_ready.to_pickle('model_ready.pickle')